In [1]:
import cv2
import numpy as np

def get_center(box):
    x, y, w, h = box
    return (x + w // 2, y + h // 2)

def find_closest_tracker(detection, trackers, threshold=60):
    x, y, w, h = detection
    detection_center = get_center((x, y, w, h))

    closest_tracker = None
    min_distance = threshold
    for tracker in trackers:
        tracker_center = get_center(tracker.last_seen)
        distance = np.linalg.norm(np.array(detection_center) - np.array(tracker_center))

        if distance < min_distance:
            closest_tracker = tracker
            min_distance = distance

    return closest_tracker

class PersonTracker:
    def __init__(self, initial_detection, color, frame, id, life=5):
        x, y, w, h = initial_detection
        self.tracker = cv2.TrackerKCF_create()
        self.tracker.init(frame, initial_detection)
        self.color = color
        self.last_seen = initial_detection
        self.life = life
        self.template = cv2.cvtColor(frame[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY)  # Guardar la plantilla
        self.id = id
    def update(self, frame):
        success, box = self.tracker.update(frame)
        if success:
            self.last_seen = box
            self.life = 5  # Restablecer vida al detectar con éxito
        else:
            self.life -= 1  # Reducir vida si no se detecta
        return success, box
        
def match_template(frame, template):
    res = cv2.matchTemplate(frame, template, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(res)
    return max_val, max_loc

haar_cascade = cv2.CascadeClassifier('haarcascade_fullbody.xml')
cap = cv2.VideoCapture("people_walking2.mp4")
max_width = 200  # Máximo ancho permitido
max_height = 200  # Máximo alto permitido
trackers = []
next_color = iter(np.random.randint(0, 255, size=(100, 3), dtype=int))
next_id = 0

def generate_unique_id():
    global next_id
    result = next_id
    next_id += 1
    return result
def generate_new_color():
    return tuple(np.random.randint(0, 255, size=(3)).tolist())
    
# Configuración para optimización
scale_factor = 0.5  # Factor de escala para reducir la resolución del frame
detection_interval = 5  # Realizar detección cada 5 frames
frame_count = 0  # Contador de frames
movement_threshold = 10
previous_detections = {}

while cap.isOpened():
    retVal, frame = cap.read()
    if not retVal:
        break

    # Reducir la resolución del frame
    frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frame_count += 1
    if frame_count % detection_interval == 0:
        bodies = haar_cascade.detectMultiScale(gray, 1.05, 13)
        for (x, y, w, h) in bodies:
            # Filtrar detecciones que son demasiado grandes
            if w > max_width or h > max_height:
                continue

            closest_tracker = find_closest_tracker((x, y, w, h), trackers)

            # Verificar si hay suficiente movimiento
            if closest_tracker is None:
                best_match_id = None
                best_match_val = 0.6  # Umbral para la mejor coincidencia
                for id, data in previous_detections.items():
                    template, _ = data
                    max_val, _ = match_template(gray, template)
                    if max_val > best_match_val:
                        best_match_val = max_val
                        best_match_id = id

                if best_match_id is not None:
                    # Reutilizar el color de la detección anterior
                    color = previous_detections[best_match_id][1]
                else:
                    color = generate_new_color()
                    best_match_id = generate_unique_id()  # Generar un nuevo ID

                new_tracker = PersonTracker((x, y, w, h), color, frame, best_match_id)
                trackers.append(new_tracker)
                previous_detections[best_match_id] = (new_tracker.template, color)


    # Actualizar y dibujar los trackers existentes
    for tracker in trackers:
        success, box = tracker.update(frame)
        if not success:
            # Intentar la comparación de plantillas si el tracker falla
            max_val, max_loc = match_template(gray, tracker.template)
            if max_val > 0.8:  # Umbral de coincidencia, ajustar según sea necesario
                x, y = max_loc
                w, h = tracker.template.shape[1], tracker.template.shape[0]
                tracker.last_seen = (x, y, w, h)
                tracker.life = 5  # Restablecer la vida
                # Dibujar un círculo en la esquina superior izquierda para marcar la detección por plantilla
                cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
                used_template_matching = True
            else:
                used_template_matching = False
        else:
            used_template_matching = False

        if success or used_template_matching:
            (x, y, w, h) = [int(v) for v in tracker.last_seen]
            cv2.rectangle(frame, (x, y), (x+w, y+h), tracker.color, 2)
            if used_template_matching:
                cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)

            # Mostrar texto debajo del rectángulo
            text = f"Persona {tracker.id + 1}"
            text_position = (x, y + h + 20)  # Colocar el texto debajo del rectángulo
            cv2.putText(frame, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)  # Texto en blanco con un grosor mayor                
            
        if success or max_val > 0.8:
            (x, y, w, h) = [int(v) for v in tracker.last_seen]
            cv2.rectangle(frame, (x, y), (x+w, y+h), tracker.color, 2)

    # Eliminar trackers cuya vida ha llegado a cero
    trackers = [tracker for tracker in trackers if tracker.life > 0]

    cv2.imshow('Frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()